# Classificação de clientes

In [2]:
%matplotlib inline

# join one or more path components intelligently
from os.path import join

# Interface com o sistema operacional
import os

# Manipulação de dataframes
import pandas as pd

# Manipulação de dados tabulares
import numpy as np

# Normalização das características
from sklearn.preprocessing import StandardScaler

# visualização de dados baseada no matplotlib
import seaborn as sns

# Esboço de gráficos
from matplotlib import pyplot as plt

## Carga e inspeção dos dados

In [3]:
# Definição dos nomes das variáveis (conforme a tabela contida no enunciado)
colnames = ['ESCT', 'NDEP', 'RENDA', 'TIPOR', 'VBEM', 'NPARC',
            'VPARC', 'TEL', 'IDADE', 'RESMS', 'ENTRADA', 'CLASSE']

In [4]:
# Leitura dos dados de treino
arquivo = './credtrain.txt'
data_train = pd.read_csv(arquivo, sep='\t', header=None, names = colnames)

# Leitura dos dados de teste
arquivo = './credtest.txt'
data_test = pd.read_csv(arquivo, sep='\t', header=None, names = colnames)

**Inspeção dos dados**

In [5]:
# Inspeção da dimensão do dataset
print(data_train.shape, data_test.shape)

(1500, 12) (577, 12)


In [6]:
# Inspeção das primeiras linhas do conjunto de treinamento
data_train.head()

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE
0,1,0,360,0,313,9,52,0,25,48,0,1
1,0,0,350,1,468,10,65,0,33,6,0,1
2,0,0,1100,0,829,9,125,0,56,48,0,1
3,0,0,3000,0,552,12,76,1,31,60,0,1
4,1,0,1000,0,809,12,111,0,24,7,0,1


In [7]:
# Inspeção das primeiras linhas do conjunto de teste
data_test.head()

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE
0,0,2,500,1,618,10,85,0,36,6,0,0
1,1,0,813,0,552,4,119,0,43,48,119,1
2,3,0,350,0,488,12,66,0,43,0,0,1
3,1,0,1530,0,381,1,398,0,28,48,0,1
4,0,0,688,1,396,10,60,0,49,72,0,1


## Pré-processamento dos dados

### Transformação de variáveis não-numéricas

É importante observar que a variável ESCT (Estado Civil) é do tipo categórica, podendo assumir 4 valores diferentes (cada valor corresponde a um estado civil). Assim, diferentemente de NDEP (onde cada valor corresponde a uma quantidade de dependentes), na variável ESCT cada valor corresponde a uma categoria. Contudo, este fato pode trazer inconsistências na criação e treinamento de modelos.

Para mitigar este problema, uma alternativa é tranformar a variável ESCT em uma variável *dummy* (variável binária). Neste sentido, cada categoria da variável ESCT corresponderá a uma variável. Visto que há 4 possíveis categorias para a variável ESCT, obteremos 4 variáveis ESCT binárias.

Uma variável *dummy* é uma variável binária utilizadas para representar categorias. Neste sentido, em um caso de uma variável com 3 ou mais categorias, recomenda-se a criação de $n-1$ dummies. Diante disso, a variável ESCT será transformada em 4 "variantes dummy", onde o valor 1 corresponderá à ocorrência de determinada categoria e o valor 0 corresponderá à não ocorrência.

In [8]:
# Aplicação no conjunto de treinamento
data_train_new = pd.get_dummies(data = data_train, 
                                prefix='ESCT', 
                                columns=['ESCT'], 
                                drop_first=True)

"""
pd.get_dummies: Convert categorical variable into dummy/indicator variables.
"""

# Inspeção das primeiras linhas
data_train_new.head()

,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE,ESCT_1,ESCT_2,ESCT_3
0,0,360,0,313,9,52,0,25,48,0,1,1,0,0
1,0,350,1,468,10,65,0,33,6,0,1,0,0,0
2,0,1100,0,829,9,125,0,56,48,0,1,0,0,0
3,0,3000,0,552,12,76,1,31,60,0,1,0,0,0
4,0,1000,0,809,12,111,0,24,7,0,1,1,0,0


In [9]:
# Aplicação da transformação sobre o conjunto de teste
data_test_new = pd.get_dummies(data = data_test, prefix='ESCT', columns=['ESCT'], drop_first=True)

"""
pd.get_dummies: Convert categorical variable into dummy/indicator variables
"""

# Inspeção das primeiras linhas
data_test_new.head()

,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE,ESCT_1,ESCT_2,ESCT_3
0,2,500,1,618,10,85,0,36,6,0,0,0,0,0
1,0,813,0,552,4,119,0,43,48,119,1,1,0,0
2,0,350,0,488,12,66,0,43,0,0,1,0,0,1
3,0,1530,0,381,1,398,0,28,48,0,1,1,0,0
4,0,688,1,396,10,60,0,49,72,0,1,0,0,0


**Separação do conjunto de dados em rótulo ($\mathrm{y}$) e features ($\mathrm{x}$)**

O rótulo ($\mathrm{y}$) corresponde ao vetor contendo a variável alvo (CLASSE), enquanto que features ($\mathrm{x}$) corresponde à matriz de dados.

In [10]:
# Transformação da variável alvo do conjunto de treinamento em vetor
y_train = np.array(data_train_new['CLASSE'])

# Inspeção das primeira linhas
y_train[:5]

array([1, 1, 1, 1, 1])

In [11]:
# Transformação da variável alvo do conjunto de teste em vetor
y_test = np.array(data_test_new['CLASSE'])

# Inspeção das primeiras linhas
y_test[:5]

array([0, 1, 1, 1, 1])

In [12]:
# Transformação do conjunto de treinamento remanescente em matriz de dados
features_name_train = list(data_train_new.columns)               # nomes das colunas
features_name_train.remove('CLASSE')                             # remove variável "CLASSE"
X_train = np.array(data_train_new.loc[:, features_name_train])   # Transformação em matriz de dados

# Inspeção da matriz resultante
X_train

array([[   0,  360,    0, ...,    1,    0,    0],
       [   0,  350,    1, ...,    0,    0,    0],
       [   0, 1100,    0, ...,    0,    0,    0],
       ...,
       [   0,  570,    0, ...,    0,    0,    0],
       [   0,  360,    0, ...,    0,    0,    0],
       [   4,  501,    1, ...,    0,    0,    0]])

In [13]:
# Transformação do conjunto de teste remanescente em matriz de dados
features_name_test = list(data_test_new.columns)               # Recuperação dos nomes das colunas
features_name_test.remove('CLASSE')                            # Remoção da variável "CLASSE"
X_test = np.array(data_test_new.loc[:, features_name_test])   # Transformação em matriz

# Inspeção da matriz resultante
X_test

array([[   2,  500,    1, ...,    0,    0,    0],
       [   0,  813,    0, ...,    1,    0,    0],
       [   0,  350,    0, ...,    0,    0,    1],
       ...,
       [   3, 1200,    0, ...,    0,    0,    0],
       [   0,  600,    0, ...,    1,    0,    0],
       [   0,  800,    1, ...,    0,    0,    0]])

### Normalização das features

Antes de iniciar o treinamento, é também necessário realizar a *normalização* das características a fim de evitar problemas decorrentes à discrepância nas ordens de grandeza das features.

In [14]:
# Criação do objeto para a padronização das features
scaler = StandardScaler()

# Ajustamento do StandardScaler ao conjunto de dados de treino e padronização dos dados de treino
X_train_norm = scaler.fit_transform(X_train)

# Transformação dos dados de teste com os parâmetros ajustados a partir dos dados de treino
X_test_norm = scaler.transform(X_test)

In [15]:
# Dimensões dos datasets
print(X_train_norm.shape, X_test_norm.shape, y_train.shape, y_test.shape)

(1500, 13) (577, 13) (1500,) (577,)


## Treinamento do modelo

In [35]:
# seu código aqui

# transformando numpy arrays em  Tensores do Pytorch
# Convertendo os datsets de treino e de tests.:

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as torch_functions

# Define loss function
mse_loss = torch_functions.mse_loss
bce_loss = torch_functions.binary_cross_entropy 

# Definindo algumas constantes
BATCH_SIZE = 50
LEARNING_RATE = 1e-5
NUM_EPOCHS = 1

# como o dtype dos inputs e outputs são diferentes, precisamos convertelos
# para um tipo único, convertendo então para float32
# dataset de treino # print(y_train_tensor.dtype)
X_train_tensor = torch.from_numpy(X_train_norm).float()
y_train_tensor = torch.from_numpy(y_train).float()

# dataset de testes
X_test_tensor = torch.from_numpy(X_test_norm).float()
y_test_tensor = torch.from_numpy(y_test).float()


# Definindo objetos tensores de dataset e o dataloader para o conj. de dados
# de treino e de testes.
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_dataset_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)


test_dataset        = TensorDataset(X_test_tensor, y_test_tensor)
test_dataset_loader = DataLoader(test_dataset, BATCH_SIZE)

# imprime algumas informações importantes na tela

for x, y in train_dataset_loader:
    print('x: {}'.format(x.shape))
    print('y: {}'.format(y.shape))
    y2 = y.unsqueeze(1)
    print('y2: {}'.format(y2.shape))
    print('')
    # print('x: {0}\ny: {1}'.format(x,y))
    value =  x.T.matmul(y2)
    print(value.shape)
    print(value)
    break



x: torch.Size([50, 13])
y: torch.Size([50])
y2: torch.Size([50, 1])

torch.Size([13, 1])
tensor([[ -0.7101],
        [ -7.7911],
        [-10.9885],
        [ -2.9503],
        [  6.5243],
        [ -5.1885],
        [  4.2683],
        [-10.5322],
        [  2.4116],
        [ -4.9095],
        [ -3.2909],
        [ -2.6205],
        [  3.4256]])


In [17]:
print(X_train_tensor.float().dtype)
print(y_train_tensor.dtype)

torch.float32
torch.float32


In [78]:
# Definindo um modelo para o problema:
import torch.nn as nn

class Model(torch.nn.Module):

    def __init__(self, input_size, hidden_size, output_size, f_loss):
        super().__init__()
        self.f_loss = f_loss
        self.hidden_layer = nn.Linear(input_size, hidden_size)

        self.output_layer = nn.Linear(hidden_size, output_size)


    def forward(self, _input):
        hidden_output = self.hidden_layer(_input)
        # Usando a função de ativação da camada oculta
        out = torch_functions.relu(hidden_output)
        # camada de saída
        out = self.output_layer(out)

        out = torch_functions.sigmoid(out)
        return out
    
    def training_step(self, x, y):
        output = self(x)
        loss = self.f_loss(output, y)

        return loss

    def evaluate(self, validation_batch):
        # outputs = [self.validation_step(x) for x in validation_batch]
        loss = []
        accuracy = []
        for x, y in validation_batch:
            kw = self.validation_step(x, y)
            loss.append(kw.get('loss'))
            accuracy.append(kw.get('accuracy'))

        
        return {
            'loss': np.array(loss).mean(),
            'accuracy': np.array(accuracy).mean()
        }

    def accuracy(self, x, y): # x, y batch
        print('--------------------')
        print('x {0} y {1}'.format(x.shape, y.shape))
        _, preds = torch.max(x, dim=1)
        print('_ {0} preds {1}'.format(_, preds))
        value = torch.tensor(torch.sum(preds == y).item() / len(preds))
        print('value {}'.format(value))

    def validation_step(self, x, y): # x, y batch
        output = self(x)
        loss = self.f_loss(output, y)
        _accuracy = self.accuracy(output, y)

        return {'loss':loss.item(), 'accuracy': _accuracy.item()}



# # Primeira camada de neurônios da rede:
# layer_model_1 = torch.nn.Linear(13, 4)
# # print(X_train_tensor.shape[1])
# # print(y_train_tensor.shape[0])
# loss = mse_loss(layer_model_1(X_train_tensor[0:4]), y_train_tensor[0:4])
# print(loss.item())

input_size = X_train_tensor.shape[1]
print(input_size)
hidden_size = BATCH_SIZE
output_size = y_train_tensor.shape[0]
print(output_size)
model = Model(13, 50, 1, mse_loss)
# loss_1 = mse_loss(model(X_train_tensor), y_train_tensor)

13
1500


In [72]:
import numpy as np 

np.array([torch.tensor([1.,2.]).numpy(), torch.tensor([3.,4.]).numpy()]).mean()

tl = [torch.tensor([1.,2.]), torch.tensor([3.,4.])]
tl = torch.tensor(tl)
torch.stack(tl)
# torch.mean(tl)
# t = torch.tensor([1.,2.])
# t.numpy()

ValueError: only one element tensors can be converted to Python scalars

In [79]:

NUM_EPOCHS = 1

def traning_model(model, epochs, learning_rate, train_loader, validation_loader, opt_func=torch.optim.SGD):
    opt = opt_func(model.parameters(), lr=learning_rate)
    
    for epoch in range(epochs):
        for x, y in train_dataset_loader:
            loss = model.training_step(x, y)

            loss.backward()

            opt.step()

            opt.zero_grad()

            # Print the progress
            # if (epoch+1) % 10 == 0:
            #     print('Epoch [{}/{}], Loss: {:.4f}'.\
            #         format(epoch+1, NUM_EPOCHS, loss.item())
            #         )
        
        # validation

        result = model.evaluate(validation_loader)
        # print('Epoch [{}/{}], Loss: {:.4f}'.\
        #             format(epoch+1, NUM_EPOCHS, loss.item())
        #             )
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['loss'], result['accuracy']))

traning_model(model, NUM_EPOCHS, LEARNING_RATE, train_dataset_loader, test_dataset_loader)

--------------------
x torch.Size([50, 1]) y torch.Size([50])
_ tensor([0.3954, 0.4230, 0.4749, 0.4558, 0.4326, 0.4094, 0.4051, 0.4388, 0.4071,
        0.3987, 0.4308, 0.4672, 0.4328, 0.4214, 0.4399, 0.3960, 0.4150, 0.3744,
        0.4409, 0.4406, 0.4453, 0.4160, 0.4652, 0.4559, 0.4126, 0.4140, 0.3742,
        0.4544, 0.4207, 0.4355, 0.4388, 0.3744, 0.4463, 0.4024, 0.4307, 0.4366,
        0.4382, 0.4097, 0.4470, 0.3753, 0.4279, 0.4163, 0.4446, 0.4653, 0.4089,
        0.4081, 0.3614, 0.4136, 0.4342, 0.4411], grad_fn=<MaxBackward0>) preds tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])
value 0.5600000023841858


AttributeError: 'NoneType' object has no attribute 'item'

In [20]:
# Verificando a loss antes do treino para efeito de comparativo:
print('mse_loss :')

loss = mse_loss(model(X_train_tensor), y_train_tensor)
print(loss)

loss = mse_loss(model(X_test_tensor), y_test_tensor)
print(loss)

print('bce_loss :')

loss = bce_loss(model(X_train_tensor), y_train_tensor.unsqueeze(1))
print(loss)

loss = bce_loss(model(X_test_tensor), y_test_tensor.unsqueeze(1))
print(loss)


mse_loss :
tensor(0.2517, grad_fn=<MseLossBackward0>)
tensor(0.2514, grad_fn=<MseLossBackward0>)
bce_loss :
tensor(0.6779, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6781, grad_fn=<BinaryCrossEntropyBackward0>)


## Validação do modelo

In [22]:
# seu código aqui

 # Predição de preços de diamantes

## Carga e inspeção dos dados

In [23]:
# seu código aqui

## Pré-processamento dos dados

In [24]:
# seu código aqui

## Treinamento do modelo

In [25]:
# seu código aqui

## Validação do modelo

In [26]:
# seu código aqui

# Classificação de imagens (Fashion MNIST)

## Carga e inspeção dos dados

In [27]:
# seu código aqui

## Pré-processamento dos dados

In [28]:
# seu código aqui

## Treinamento do modelo

In [29]:
# seu código aqui

## Validação do modelo

In [30]:
# seu código aqui